## Objective

BPR을 이용한 Matrix Factorization은 효과적이나, 고객의 취향 행렬과 제품의 특성 행렬 간의 선형 관계만을 파악할 수 있다는 점에서 한계가 있습니다. 좀 더 복잡한 고객의 취향 <-> 특성 행렬 간 관계를 파악하기 위해 딥러닝을 Matrix Factorization에 적용하는 연구가 진행되었습니다. 이 중 대표적인 모델이 바로 아래 소개된 `Neural Collaborative Filtering`입니다.

* Code Reference : [Paper's Implementation](https://github.com/hexiangnan/neural_collaborative_filtering/blob/master/MLP.py)

### 필요 모듈 가져오기

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
tqdm.pandas()
np.set_printoptions(5,)

### 데이터 가져오기 

In [2]:
from tensorflow.keras.utils import get_file

ROOT_URL = "https://craftsangjae.s3.ap-northeast-2.amazonaws.com/data/"

# 데이터 가져오기
play_path = get_file("lastfm_play.csv",
                     ROOT_URL+"lastfm_play.csv")
artist_path = get_file("lastfm_artist.csv",
                       ROOT_URL+"lastfm_artist.csv")
user_path = get_file("lastfm_user.csv",
                     ROOT_URL+"lastfm_user.csv")

play_df = pd.read_csv(play_path)
artist_df = pd.read_csv(artist_path)
user_df = pd.read_csv(user_path)

##  Neural Collaborative Filtering 구현하기

<img src="https://i.imgur.com/B0lLZEg.png" width="400">

Neural Collaborative Filtering은 이 구조도로 모델링을 설명할 수 있습니다.  유저의 임베딩 값과 아이템의 임베딩 값을 딥러닝의 Input으로 들어가게 됩니다. 그리고 4층을 걸쳐 출력값으로 고객이 해당 아이템을 선호할 확률이 나오게 됩니다.

### Input 구성하기

우선 user와 item이 쌍으로 Input으로 들어가게 됩니다.

In [3]:
from tensorflow.keras.layers import Input

user_id = Input(shape=(), name='user')
item_id = Input(shape=(), name='item')

### 임베딩 레이어 구성하기

임베딩의 크기는 논문에서 알려져 있듯, 값이 커질수록 그 성능이 올라갑니다. 여기에서는 32로 두도록 하겠습니다.

In [6]:
from tensorflow.keras.layers import Embedding

num_user = play_df.user_id.max() + 1
num_item = play_df.artist_id.max() + 1
num_factor = 32

user_embedding = Embedding(num_user, num_factor)(user_id)
item_embedding = Embedding(num_item, num_factor)(item_id)

### NCF Layers 구성하기

`Dense` 레이어가 순서대로 적층되어 있는 단순한 구조를 띕니다. 논문에서 서술되어 있는 방식으로, 3층 구조로 unit수가 ( 32 $\rightarrow$ 16 $\rightarrow$ 8 )로 줄어드는 순서로 작성하였습니다.

In [7]:
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dense

concat_embedding = Concatenate()(
    [user_embedding, item_embedding])

hidden1 = Dense(32, activation='relu')(concat_embedding)
hidden2 = Dense(16, activation='relu')(hidden1)
hidden3 = Dense(8, activation='relu')(hidden2)
probs = Dense(1, activation='sigmoid')(hidden3)

### (4) Model 구성하기

입력값은 `user_id`와 `item_id`가 Pair로 들어가게 되고, 출력값은 `user_id`가 `item_id`를 선호할 확률이 계산됩니다.  `user_id`가 `item_id`를 구매하거나 산 적이 있으면 1이 나와야 하고, `user_id`가 `item_id`를 구매하거나 산 적이 없다면 0이 나아야 합니다.


In [8]:
from tensorflow.keras.models import Model

model = Model([user_id, item_id],  probs, name='NCF')

논문에서 학습하기 위해 `Adam` 옵티마이저를 이용하였고, 손실함수로서는 `BinaryCrossentropy`를 이용하였습니다.

In [9]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

model.compile(Adam(1e-3), 
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

## 모델 학습하기

Bayesian Personalized Ranking과 마찬가지로, 데이터는 오로지 고객이 평가한 영화에 대한 정보만 존재합니다. 모델을 학습하기 위해서, 고객이 평가하지 않은 영화에 대한 정보 또한 필요합니다. 보지 않은 영화 모두를 가져오게 되면 지나치게 많은 데이터가 생기므로 그중에서 일부분만 샘플링을 통해 추출하게 됩니다.

### 데이터 파이프라인 : Negative Sampling 수행하기

NCF 모델은 user, item의 쌍으로 추론합니다. 해당 User가 해당 Item을 선호할 확률을 추론하기 때문에, 실제로 클릭 혹은 구매한 적이 있으면 학습 데이터의 라벨로는 1을 주고 클릭한적이 없다면 학습 데이터의 라벨로는 0을 줍니다. 유저가 클릭하지 않은 아이템을 Negative Sample이라 하고, 우리는 Negative Sample을 만들어주어야 합니다. <br>

NCF 모델에서 Positive Sample 대비 Negative Sampling의 비율을 보통 3.~6. 사이로 두는 것을 권고합니다.

### 모델 학습시키기

epoch 10번에 걸쳐, 모델을 학습시키도록 하겠습니다. 매 Epoch마다 새로운 학습 데이터를 생성하도록 하였습니다. 

In [ ]:
num_epoch = 10
batch_size = 1024
for i in range(epoch):
    print(f"{i+1}th epoch :")
    dataset = 
    model.fit(dataset, verbose=2)

## 모델 평가하기

Bayesian Personalized Ranking 대비 성능을 비교하기 위해 Hit Ratio를 산출해보도록 하겠습니다.

In [ ]:
hit = 0.
for i, row in tqdm(hit_ratio_df.iterrows()):
    user = np.array([row.user_id])
    seens = np.array([row.item_id])
    pos_scores = model.predict([user,seens])
    pos_scores = pos_scores[0,0]
    
    not_seens = np.array(row.not_seen_list)
    users = np.array([row.user_id]*len(not_seens))   
    neg_scores = model.predict([users,not_seens])
    
    if pos_scores > np.sort(neg_scores.flatten())[-10]:
        hit += 1

In [ ]:
hit_ratio = hit / len(hit_ratio_df)        
print(f"hit ratio : {hit_ratio:.3f}")